In [1]:
# scrape data and append it in list of dictionary

def scrape_current_listing():
    
    from bs4 import BeautifulSoup # use BeautifulSoup to send request
    
    global chrome
    
    soup = BeautifulSoup(chrome.page_source, 'html.parser') # send request

    listing = {'url': chrome.current_url} # link of iten

    parent_tag = soup.find('div', class_ = 'head-content')

    if parent_tag: # check if information in page

        tag = parent_tag.div 
        if tag:
            listing['brand-alt'] = tag.text.strip()

        tag = parent_tag.h1
        if tag:
            listing['title'] = parent_tag.h1.text.strip()

        tag = parent_tag.find('span', id = 'item_id')
        if tag:
            listing['id'] = tag.text.strip()

        tag = parent_tag.find('span', class_ = 'total_review')
        if tag:
            listing['rating'] = tag.text.strip()

        tag = parent_tag.find('span', itemprop = 'price')
        if tag:
            listing['price'] = tag['content']
        else:
            print('Debug: price not found...')
#             raise Exception('Debug: price not found...')
            
    tag = soup.find('span', class_ = 'rating-count')
    if tag:
        listing['rating-count'] = tag.text.strip()

    tag = soup.find('span', id = 'availability-status')
    if tag:
        listing['availability'] = tag.text.strip()

    tag = soup.find('div', class_ = 'product-store')
    if tag:
        listing['product-store'] = tag.text.strip()

    tag = soup.find('select', id = 'qty')
    if tag:
        listing['quantity'] = tag.option['value']

    tag = soup.find('ul', itemprop = 'description')
    if tag:
        tags = tag.find_all('li')
        listing['features'] = '; '.join([x.text.strip() for x in tags])

    tag = soup.find('table', class_ = 'featureTable')
    if tag:
        
        tags = tag.find_all('tr')
        temp_dict = dict([[y.text.strip() for y in x.find_all('td')] for x in tags if len(x.find_all('td')) == 2])
        listing.update(temp_dict)
        
        tag = tag.p
        if tag:
            listing['brief'] = tag.text.strip()
        
    else:
        print('Debug: feature-table not found...') #if Feature table found in page or not and save it in dictionary
#         raise Exception('Debug: feature-table not found...')
        
    tag = soup.find('div', class_ = 'product-description')
    if tag:
        tags = tag.find_all('tr')
        temp_dict = dict([[y.text.strip() for y in x.find_all('td')] for x in tags if len(x.find_all('td')) == 2])
        listing.update(temp_dict)
        
    return listing

def scrape_and_append_current_listing(): # append dectionary in list to save it
    
    global listings
    
    listing = scrape_current_listing()
    listings.append(listing)
    

# append list then check if this product is scraping or no
# if is exite
# else continue
def scrape_and_append_current_listing_variations(): 
    
    from bs4 import BeautifulSoup
    from selenium.webdriver.support.ui import Select # to select options
    from time import sleep #to to make sure all informations is loaded
    import itertools # to choice select list
    from random import sample # to random choice
    
    global n_configs
    global get_delay_s

    soup = BeautifulSoup(chrome.page_source, 'html.parser')
    tag = soup.find('div', class_ = 'product-variations')
    if tag:

        select_tags = tag.find_all('select')
        
        labels = [x.text.strip() for x in tag.find_all('label')]
        options = [[y.text.strip() for y in x.find_all('option')] for x in select_tags] # all select option

        select_ids = [x['id'] for x in select_tags]
        select_options = [list(range(len(x.find_all('option')))) for x in select_tags]
        configs = list(itertools.product(*select_options))
        configs = [dict(zip(select_ids, x)) for x in configs]
        
        # to mack shore no error in number of choice option
        if n_configs < len(configs):
            configs = sample(configs, n_configs)
        
        for config in configs:
            for x in config:
                select_element = Select(chrome.find_element(by = 'id', value = x))
                select_element.select_by_index(config[x])
            sleep(get_delay_s)
            listing = scrape_current_listing()
            for i, x in enumerate(config):
                label = labels[i]
                option_index = config[x]
                listing[label] = options[i][option_index]
            print(listing)
            listings.append(listing)
            get(chrome.current_url)
    
    else:
        
        scrape_and_append_current_listing()
        
        
# scroll down all the page to load all data of item             
def scroll_down_all_the_way():
    
    from time import sleep
    
    global chrome
    global scroll_delay_s
    
    end = 0
    while end < chrome.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 1000
        chrome.execute_script(f'window.scrollTo({start}, {end})')
        sleep(scroll_delay_s)

# to get thne page link and all laptops link of it
def scrape_current_search_results():
    
    from bs4 import BeautifulSoup
    import re
    
    global start_url
    
    soup = BeautifulSoup(chrome.page_source, 'html.parser')
    
    tag = soup.find('div', id = 'usstore-products')
    tags = tag.find_all('div', class_ = re.compile('product-inner-list$'))
    listing_urls = [x.find('a', class_ = re.compile('^img-detail'))['href'] for x in tags]

    tag = soup.find('li', title = 'Next')
    nextpage_index = tag.a['data-pageno']
    nextpage_url = '='.join((start_url.split('=')[:-1] + [nextpage_index]))
    
    return nextpage_url, listing_urls


# append all links to scraping it
def scrape_and_append_current_search_page_urls():
    
    global url_stack
    
    nextpage_url, listing_urls = scrape_current_search_results()
    url_stack.append(nextpage_url)
    url_stack.extend(listing_urls)


# save all this operations if it successful
def load_previous_state_if_exists():
    
    import joblib
    
    global state_filepath #location of this file in PC
    global start_url
    
    try:
        return joblib.load(state_filepath)
    except:
        return [], [start_url]

# save all this operations in PC
def save_state():
    
    import joblib
    
    global listings
    global url_stack
    global state_filepath
    
    state = listings, url_stack
    joblib.dump(state, state_filepath)

# to open all links in the browser
def get(url):
    
    from time import sleep
    from selenium.common.exceptions import TimeoutException
    
    global chrome
    global get_delay_s
    global timeout_delay_s
    
    while True:
        try:
            chrome.get(url)
            sleep(get_delay_s)
            break
        # to if the user want to interaote kernal
        except TimeoutException:
            print('timeout detected... cooling off...')
            sleep(timeout_delay_s)

# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
    
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    chromedriver = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
    
    return chromedriver

# to get number of last page that we scraping
def check_current_page_type():
    
    global chrome
    
    is_search_page = chrome.current_url.split('/')[-1].startswith('laptops')
    
    if is_search_page:
        
        return 'search-results'
    
    return 'listing'

#global variables

n_configs = 16
get_delay_s = 60.0
timeout_delay_s = 120.0
scroll_delay_s = 7.0
state_filepath = r'E:\other\Epsilon_Ai\Final Project\final project\state.pkl'
start_url = 'https://www.ubuy.com.eg/en/category/electronics/laptops-565108?page=1'

In [2]:
listings, url_stack = load_previous_state_if_exists()

chrome = open_chromedriver()

while url_stack: # to git all links
    
    current_url = url_stack.pop() # to search about link by link
    
    scraped_listing_urls = {x['url'] for x in listings}
    
    if current_url not in scraped_listing_urls: # check if craping it or not 
        
        print(len(listings), current_url)
        
        get(current_url)
        
        if check_current_page_type() == 'search-results':

            scroll_down_all_the_way()
            scrape_and_append_current_search_page_urls()

        else:

            scrape_and_append_current_listing_variations()
            save_state()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_17368/48173510.py:229: DeprecationWarning: use options instead of chrome_options
  chromedriver = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


1497 https://www.ubuy.com.eg/en/product/48Z17GEOK-microsoft-surface-pro-x-13-wi-fi-tablet-microsoft-sq2-16gb-ram-512gb-ssd-platinum-microsoft-sq2-processor-laptop-tablet-or-studio-mode
1498 https://www.ubuy.com.eg/en/product/1GC38ZF04-dell-latitude-e5470-laptop-core-i5-6200u-2-3ghz-8gb-256gb-ssd-14in-hd-window-10-pro-64bit-manufacturer-refurbished
1499 https://www.ubuy.com.eg/en/product/48YZY4VSE-dell-g15-15-6-full-hd-1920-x-1080-gaming-laptop-amd-ryzen-7-5800h-8gb-ram-512gb-ssd-nvidia-geforce-rtx-3050-ti-graphics-windows-11-home
1500 https://www.ubuy.com.eg/en/product/1I88Y5F8G-dell-latitude-e7470-laptop-core-i5-6300u-2-4ghz-8gb-256gb-ssd-14in-hd-window-10-pro-64bit-manufacturer-refurbished
1501 https://www.ubuy.com.eg/en/product/1AETL5EQM-lenovo-thinkpad-x260-laptop-core-i5-6200u-2-3ghz-6th-gen-processor-8gb-memory-240gb-ssd-win-10-pro-64-bit-manufacturer-refurbished
1502 https://www.ubuy.com.eg/en/product/1I1RRQIFY-lenovo-thinkpad-t440-laptop-core-i5-4300u-1-9ghz-8gb-256gb-ssd-14in-

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="select_ram"]"}
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CDB8F3+2406643]
	Ordinal0 [0x00C6AF31+1945393]
	Ordinal0 [0x00B5C748+837448]
	Ordinal0 [0x00B892E0+1020640]
	Ordinal0 [0x00B8957B+1021307]
	Ordinal0 [0x00BB6372+1205106]
	Ordinal0 [0x00BA42C4+1131204]
	Ordinal0 [0x00BB4682+1197698]
	Ordinal0 [0x00BA4096+1130646]
	Ordinal0 [0x00B7E636+976438]
	Ordinal0 [0x00B7F546+980294]
	GetHandleVerifier [0x00F49612+2498066]
	GetHandleVerifier [0x00F3C920+2445600]
	GetHandleVerifier [0x00D74F2A+579370]
	GetHandleVerifier [0x00D73D36+574774]
	Ordinal0 [0x00C71C0B+1973259]
	Ordinal0 [0x00C76688+1992328]
	Ordinal0 [0x00C76775+1992565]
	Ordinal0 [0x00C7F8D1+2029777]
	BaseThreadInitThunk [0x75F16739+25]
	RtlGetFullPathName_UEx [0x77888E7F+1215]
	RtlGetFullPathName_UEx [0x77888E4D+1165]


In [6]:
import pandas as pd
import joblib


listings = joblib.load(r"E:\other\Epsilon_Ai\Final Project\final project\state.pkl") # load data form PC
pd.options.display.max_columns = None # donot display all rows and columns of data

In [9]:
df = pd.DataFrame.from_records(listings[0]) # show result
df
# df.to_csv(r"E:\other\Epsilon_Ai\Final Project\final project\state.csv")

,url,brand-alt,title,id,rating,price,rating-count,availability,product-store,quantity,Overview,Colour,Manufacturer Color,Processor Brand,Hard Drive Capacity,Processor Core Type,Manufacturer,Product Line,Condition,Processor Speed,Manufacturer Part Number,Wireless Technology,RAM Memory,Model,Screen Size,Brand,Features,Assembled Product Dimensions (L x W x H),Actual Color,Processor Type,Operating System,Battery Life,Maximum RAM Supported,Video Game Platform,Material,features,CPU,RAM Size,Hard Disk Size,Graphics Description,Color,Capacity,Style,Pattern Name,Size,Configuration,Display Size,manufacturer_part_number,Edition,SSD Capacity,Graphics processor,Assembled Product Weight,RAM for Multitasking,SSD for Fast Running,Standing screen display size,Screen Resolution,Max Screen Resolution,Processor,RAM,Memory Speed,Hard Drive,Graphics Coprocessor,Chipset Brand,Card Description,Graphics Card Ram Size,Wireless Type,Number of USB 2.0 Ports,Number of USB 3.0 Ports,Series,Item model number,Hardware Platform,Item Weight,Product Dimensions,Item Dimensions LxWxH,Processor Count,Computer Memory Type,Flash Memory Size,Hard Drive Interface,Optical Drive Type,Voltage,Batteries,ASIN,CPU Model,Ram Memory Installed Size,Date First Available,Dimensions (Overall),Weight,Battery Charge Life,Electronics Condition,Aspect Ratio,Data Storage Drive Capacity,Maximum Resolution,Video Recording Resolution,Data storage type,Megapixels,System RAM,Model name,Processor model,Battery,Hard Drive Speed,Number of USB ports,Drive Type,Primary use,Memory RAM Type,Graphics card model,Number of cores,Data storage capacity,Microphone,Touchscreen,Backlit Keyboard,Overall Width,UPC,Graphics Processing Type,MPN,Most Suitable For,Type,Release Year,Connectivity,Storage Type,GPU,Operating System Edition,Optical Drive,EAN,Warranty - Labor,Manufacturer Warranty,Warranty - Parts,Country/Region of Manufacture,Modified Item,Item Height,Average Battery Life (in hours),Item Length,Item Width,Custom Bundle,PointingDevice,Bluetooth,NetworkCard,Processor Type (cont.),Bundle Listing,Memory,Hardware Connectivity,Product Name,WirelessWAN,SSD,Processor Max Turbo Speed,RAM For Multi Tasking,SSD For Fast Running,Compatible Laptop Size,Maximum load weight,Item Color
0,https://www.ubuy.com.eg/en/product/4SUMCH0KE-n...,Generic,NEW HP 2560P i5-2540M 2.6GHZ 4GB 128GB SSD Dis...,49792689,0 rating,5923,0,In stock,US Store,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.ubuy.com.eg/en/product/3IN7HY-hp-1...,Generic,"HP 15.6"" HD - AMD Ryzen 3 - 8GB RAM / 256GB SS...",25497033,0 rating,11200,0,In stock,US Store,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.ubuy.com.eg/en/product/48929YHEU-n...,Generic,"MSI Modern 14"" Ultrabook Laptop Intel Core i5-...",47553427,0 rating,14533,0,In stock,US Store,1,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na